In [1]:
#installera pandas och matplotlib, 
# och numpy om den inte kommer automatiskt med pandas
import json
import glob
import os
import re
import pandas as pd
import numpy as np
import time
import pprint
import matplotlib.pyplot as plt
import datetime
import time
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#Printar lite extra grejs som kan vara intressant om saker går fel
DEBUG = False

In [ ]:
#Genererar ett filträd. Måste ligga i samma mapp som datamapparna
# tree[date][industry][company][handle] är en lista med alla filer för den användaren (alltså: mentions, till, från).  

#TESTAD 19/3, Patrik har kontrollerat att längden på fillistan stämmer överrens med antal jsonfiler och att filnamn
# inte förekommer dubbelt. Koden är testad när notebooken ligger i samma mapp som två datamappar (2020-03-18, 2020-03-19).


tree = {}
INDUSTRIES = ["Airlines"]

#Lista med alla mappar i working directory som heter något i stil med 1111-11-11
collections = [d for d in glob.glob("*") if re.fullmatch(r'[0-9]{4}-[0-9]{2}-[0-9]{2}', d)]

for col in collections:
    tree[col] = {}
    for ind in INDUSTRIES:
        tree[col][ind] = {}
        
        #Den här fångar alla undermappar i /2020-xx-xx/<industry>/
        companies = [os.path.basename(x) for x in glob.glob(os.path.join(col, ind, "*"))]
        
        for comp in companies:
            tree[col][ind][comp] = {}
            
            #Traskar igenom "underfilträdet" för varje företag, alltså med /2020-xx-xx/industry/company/ som root. 
            #Lite snårig men finns bra dokumentation online.
            for root, dirs, files in os.walk(os.path.join(col, ind, comp), topdown=True):
                if dirs:
                    for handle in dirs:
                        tree[col][ind][comp][handle] = []
                else:
                    for f in files:
                        if f.find("tweet") == 0:
                            tree[col][ind][comp][os.path.basename(root)].append(os.path.join(root, f))
                            
#Printar en översikt av hela filträdet
if DEBUG:
    for c in tree.keys():
        print(c)
        for ind in tree[c].keys():
            print('\t', ind)
            for comp in tree[c][ind].keys():
                print('\t\t', comp)
                for handle in tree[c][ind][comp].keys():
                    print('\t\t\t', handle)
                    print('\t\t\t', "antal json:", str(len(tree[c][ind][comp][handle])))
                    for f in tree[c][ind][comp][handle]:
                        print('\t\t\t\t', f)

In [ ]:
#Funktion som analyserar och returnerar sentimentet för en tweet
def analyse_sentiment(tweet_str):
    #Skapar objektet som utför sentimentanalysen
    analyzer = SentimentIntensityAnalyzer()
    #Analyserar sentimentet
    sentiment = analyzer.polarity_scores(tweet_str)
    #Returnerar det sammanställda sentimentet, där både positivt, negativt och neutralt sentiment har räknats in samt eventuella ord/punktuationer/liknande som förstärker känslan åt något håll.
    return sentiment["compound"]

In [ ]:
#Läser in allt i flera DataFrames och skriver ut hur lång tid det tog.
#ITEMS är de fält som tas med. Filtrerar direkt för att spara minne.
ITEMS = ["created_at", "full_text", "id", "id_str", "metadata", "in_reply_to_status_id", "user", "retweeted_status", "in_reply_to_screen_name"]

#dictionary för alla DataFrames
frames = {}

with open("companies_europe.json") as f:
    json_list = json.load(f)[0]["companies"]
    companies = [company["tweet_handle"] for company in json_list]
for company in companies:
    frames[company] = [] 
t_start = time.time()
for day in tree.keys():
    for company in tree[day]["Airlines"].keys():
        for path in list(tree[day]["Airlines"][company].values())[0]:
            df = pd.read_json(path)
            #Ta bort alla fält som inte finns i items
            df = df.filter(items=ITEMS)
            #Ta bort retweets
            df = df[df["retweeted_status"].isnull()]
            #Skapa en bool-series baserat på språket och behåll endast kolumner där språket känns igen som engelska.
            en = df.metadata.apply(lambda x: x["iso_language_code"] == "en")
            df = df[en]
            #Plocka ut id och screen name från user, de får egna kolumner
            df["user.id"] = df.user.apply(lambda x: x["id"])
            df["user.screen_name"] = df.user.apply(lambda x: x["screen_name"])
            #Lägg till kolumner som anger vilket företag tweeten insamlats från (alltså EJ avsändaren), samt insamlingsdatum
            df["associated_company"]=company
            df["collection_date"]=day
            #Ta bort kolumner vi inte använder
            df = df.drop(["metadata", "user", "retweeted_status"], axis=1)
            df["sentiment"] = df.full_text.apply(lambda x: analyse_sentiment(x))
            #Lägg till i dictionaryt
            frames[company].append(df)
    
print("Time - read files:", time.time()-t_start, "seconds")



In [ ]:
#Styr upp all inläst data.
#Slår ihop alla listor i frames så att varje företag får en egen DataFrame. 
# -Alla tweets i ett företags sammanslagna DataFrame är unika. 
# -Samma tweet kan förekomma i två olika företags DataFrames 
#       (om de tex har taggat flera företag eller svarat ett företag och taggat ett annat)
#Skapar även en DataFrame all_data där alla DataFrames från olika företag slås ihop
# -Alla tweets i all_data är unika.
# -Kolumnen "associated_company" i all_data är alltså oren eftersom tweets som taggar flera företag här endast
#    tillhör det företag som programmet såg först.

#DUPLICATES SYFTAR ALLTSÅ PÅ ATT TWEETS SOM TAGGAR FLERA FÖRETAG RÄKNAS DUBBELT, INTE ATT DE SAMLATS IN DUBBELT

#totala antalet tweets i dictionaryt frames
total_dup = 0
#totala minnesanvändningen för frames + all_data
total_memory = 0
all_data = []
for company in companies:
    #slår ihop alla DataFrames för ett företag till en enda och tar bort dubbelinsamlade tweets för det företaget.
    frames[company] = pd.concat(frames[company])
    frames[company] = frames[company].drop_duplicates(subset="id")
    #Lägger till den framen till all_data
    all_data.append(frames[company])
    
    total_dup += frames[company].shape[0]
    mem = frames[company].memory_usage(index=True, deep=True).sum() / 1000000
    total_memory += mem
    #Skriver ut antalet unika tweets för varje företag och hur mycket minne som går åt (mest för kontroll)
    print(company, "- tweets:", frames[company].shape[0], "- memory:", mem)
#Slår ihop all_data till en frame och tar bort duplicates
all_data = pd.concat(all_data)
all_data = all_data.drop_duplicates(subset="id")
all_data = all_data.astype('int64')
all_data = all_data.set_index("id")
mem = all_data.memory_usage(index=True, deep=True).sum() / 1000000
total_memory+=mem
#Skriver ut totala antalet unika tweets och hur mycket minne all_data tar
print("all_data - tweets:", all_data.shape[0], "-memory:", mem)
#Skriver ut summan av antalet tweets för varje företag, samt totala minnesanvändningen (frames + all_data)
#Se ovan exakt vad som menas med duplicates.
print("total (with duplicates) - tweets:", total_dup, "- memory:", total_memory)

In [11]:
all_data.head()

,created_at,full_text,id_str,in_reply_to_status_id,in_reply_to_screen_name,user.id,user.screen_name,associated_company,collection_date,sentiment
id,,,,,,,,,,
1240031660302905349,2020-03-17 21:46:01+00:00,@lufthansa does your call charges take place w...,1240031660302905344,NaN,lufthansa,53153541,B1GSTEW1E,Lufthansa Group,2020-03-18,-0.5423
1240031502395678720,2020-03-17 21:45:23+00:00,@Robert_Albornoz Since we don't handle rebooki...,1240031502395678720,1.240031e+18,Robert_Albornoz,124476322,lufthansa,Lufthansa Group,2020-03-18,0.3182
1240031223315083265,2020-03-17 21:44:16+00:00,@lufthansa Hi @lufthansa. Thanks for your answ...,1240031223315083264,1.240029e+18,lufthansa,79372015,Robert_Albornoz,Lufthansa Group,2020-03-18,0.3997
1240030557586763776,2020-03-17 21:41:38+00:00,@lufthansa Please can get a reply on how shoul...,1240030557586763776,1.239541e+18,RichaJo34474468,1235799034033471488,RichaJo34474468,Lufthansa Group,2020-03-18,0.3182
1240030377072300033,2020-03-17 21:40:55+00:00,"@lufthansa I can't rebook, or do anything onli...",1240030377072300032,1.238504e+18,lufthansa,81614898,chavax,Lufthansa Group,2020-03-18,-0.2500


In [17]:
print(frames["Lufthansa Group"])

created_at  \
1   2020-03-17 21:46:01+00:00   
3   2020-03-17 21:45:23+00:00   
5   2020-03-17 21:44:16+00:00   
6   2020-03-17 21:41:38+00:00   
7   2020-03-17 21:40:55+00:00   
..                        ...   
990 2020-03-17 22:00:39+00:00   
991 2020-03-17 21:58:52+00:00   
995 2020-03-17 21:51:46+00:00   
997 2020-03-17 21:51:39+00:00   
999 2020-03-17 21:48:35+00:00   

                                             full_text                   id  \
1    @lufthansa does your call charges take place w...  1240031660302905349   
3    @Robert_Albornoz Since we don't handle rebooki...  1240031502395678720   
5    @lufthansa Hi @lufthansa. Thanks for your answ...  1240031223315083265   
6    @lufthansa Please can get a reply on how shoul...  1240030557586763776   
7    @lufthansa I can't rebook, or do anything onli...  1240030377072300033   
..                                                 ...                  ...   
990  @lufthansa I am stuck in Colombia - my flight ...  1240035342796

In [19]:
# Save to csv file
all_data.to_csv('all_data.csv')

In [43]:
# Read csv file
all_data = pd.read_csv("all_data.csv", dtype={"id": np.int64})
all_data.head(10)

,id,created_at,full_text,id_str,in_reply_to_status_id,in_reply_to_screen_name,user.id,user.screen_name,associated_company,collection_date,sentiment
0,1240031660302905349,2020-03-17 21:46:01+00:00,@lufthansa does your call charges take place w...,1240031660302905344,NaN,lufthansa,53153541,B1GSTEW1E,Lufthansa Group,2020-03-18,-0.5423
1,1240031502395678720,2020-03-17 21:45:23+00:00,@Robert_Albornoz Since we don't handle rebooki...,1240031502395678720,1.240031e+18,Robert_Albornoz,124476322,lufthansa,Lufthansa Group,2020-03-18,0.3182
2,1240031223315083265,2020-03-17 21:44:16+00:00,@lufthansa Hi @lufthansa. Thanks for your answ...,1240031223315083264,1.240029e+18,lufthansa,79372015,Robert_Albornoz,Lufthansa Group,2020-03-18,0.3997
3,1240030557586763776,2020-03-17 21:41:38+00:00,@lufthansa Please can get a reply on how shoul...,1240030557586763776,1.239541e+18,RichaJo34474468,1235799034033471488,RichaJo34474468,Lufthansa Group,2020-03-18,0.3182
4,1240030377072300033,2020-03-17 21:40:55+00:00,"@lufthansa I can't rebook, or do anything onli...",1240030377072300032,1.238504e+18,lufthansa,81614898,chavax,Lufthansa Group,2020-03-18,-0.2500
5,1240030350790873103,2020-03-17 21:40:48+00:00,@lufthansa I booked return flights and one fli...,1240030350790873088,NaN,lufthansa,1238521313725120518,Sarah56855438,Lufthansa Group,2020-03-18,-0.2263
6,1240030169773101059,2020-03-17 21:40:05+00:00,@Holgate1987 @lufthansa But I can't get throug...,1240030169773101056,1.239938e+18,Holgate1987,40105250,katsjourney,Lufthansa Group,2020-03-18,0.0000
7,1240029898716176387,2020-03-17 21:39:01+00:00,When you fly for peanuts with @easyJet it's fa...,1240029898716176384,NaN,NaN,499250971,Sussextwinchick,Lufthansa Group,2020-03-18,0.0000
8,1240029788498046976,2020-03-17 21:38:34+00:00,"@agenteo @lufthansa Hey,\nI clicked on button ...",1240029788498046976,1.240028e+18,agenteo,983917577494642688,PhilipRegin,Lufthansa Group,2020-03-18,-0.7096
9,1240029521333682176,2020-03-17 21:37:31+00:00,Update: @lufthansa have cancelled all my fligh...,1240029521333682176,1.239832e+18,RogerIDnow,743018625150377984,RogerIDnow,Lufthansa Group,2020-03-18,0.4939


In [44]:
all_data.dtypes
all_data = all_data.set_index("id")

In [98]:
#Rekursiv funktion som bygger upp ett dictionary med tweet-id:s baserat på tweet-trådar, konversationer. 
#Varje key är ett id, och varje value är antingen "None", om det inte finns några svar på tweeten, eller ett nytt dictionary med tweetsvarets id som key och antingen "None" eller nytt dictionary som value. Så fortsätter det tills konversationen är slut. 
def build_threads(all_tweets, initial_tweets):
    #Skapar nytt dictionary med key: id och value: None
    subthreads = dict.fromkeys(initial_tweets.index, None) 
    
#För varje tweet i initial_tweets kollas vilka tweets (bland alla tweets) som har med dess id (indexet) i sina in_reply_to-fält. Detta är svar på initial-tweetsen, och sparas i "responses". Så länge responses inte är tom, anropas funktionen igen för att denna gång leta reda på vilka svar som finns på tweetsen i responses. 
    for index in initial_tweets.index:
        if index in subthreads.keys():
            responses = all_tweets[all_tweets.in_reply_to_status_id == index]
            if not responses.empty:
                subthreads[index] = build_threads(tweets, responses)
#Returnerar antingen ett nested dictionary, om det funnits svar, eller ett dictionary med de inskickade tweetsens id som keys och "None" som value. 
    return subthreads

In [106]:
#Konversationträdet byggs upp. 
t_start = time.time()
#Filtrerar bort onödiga kolumner för en något snabbare körning 
tweets = all_data.filter(items=["in_reply_to_status_id"]) 
#Hittar de tweets som påbörjar konversationen
initial_tweets = tweets[tweets.in_reply_to_status_id.isnull()] 
#Bygg trädet! 
threads = build_threads(tweets, initial_tweets)

print("Time - build conversation threads:", time.time()-t_start), "seconds"
print("Antalet konversationer, inkl. de med 0 svar:", len(threads.keys()))
print("Antalet konversationer med > 0 svar:", len([x for x in threads.values() if x is not None]))
print("Memory:", sys.getsizeof(threads)/1000000, "MB")

Time - build conversation threads: 93.40913796424866
Antalet konversationer, inkl. de med 0 svar: 51837
Antalet konversationer med > 0 svar: 12459
Memory: 2.621544 MB


In [107]:
conversations = {k: v for k, v in threads.items() if v is not None}

In [116]:
threads = {}
conversations = {}
for company in all_data["associated_company"].unique():
    company_tweets = all_data[all_data["associated_company"] == company]
    company_tweets = company_tweets.filter(items=["in_reply_to_status_id"]) 
    initial_tweets = company_tweets[tweets.in_reply_to_status_id.isnull()]
    company_threads = build_threads(company_tweets, initial_tweets)
    company_conversations = {k: v for k, v in company_threads.items() if v is not None}
    threads[company] = company_threads
    conversations[company] = company_conversations
    #threads.append((company, company_threads))
    #conversations.append((company, company_conversations))


In [117]:
#Skapar en pretty printer för att kunna printa dictionaries, och printar de 100 första twittertrådarna 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(dict(list(conversations.items())[2:4])) 

{   'British Airways': {   1238522332127170561: {1239132224857034752: None},
                           1238522806029221888: {   1238523088393904130: {   1238534865957650432: None}},
                           1238523663328129025: {1239559943621169153: None},
                           1238524778627170306: {   1238835597109796864: None,
                                                    1238840994210160640: {   1238849172473892866: {   1238849654382563329: {   1239178197343907846: {   1239180698214416385: {   1239186479093514240: {   1239187763800481801: None}}}}}}},
                           1238525095741722624: {   1238534943724142592: {   1238535352475947010: None,
                                                                             1238726073875128321: None}},
                           1238525511917350918: {1239625897902329857: None},
                           1238525656578826242: {   1238525894924402690: {   1238526284201889794: None},
                                 

In [46]:
all_data['created_at'] = pd.to_datetime(all_data['created_at'])

In [4]:
def calculate_mean(newdata, avg, count):
    if count != 0:
        avg = avg + (newdata - avg)/count
    return avg

In [47]:
with open("companies_europe.json") as f:
    json_list = json.load(f)[0]["companies"]
    companies = [company["twitter_handle"] for company in json_list]
    reply_times = {}

for company in companies:
    company_tweets = all_data[all_data["user.screen_name"] == company]
    
    if(company_tweets.empty):
        continue
    
    company_tweets = company_tweets[company_tweets["in_reply_to_status_id"].notnull()]
    mean_reply_time = datetime.timedelta(0,0,0,0,0,0)
    counts = 0

    company_reply_time = {}
    for date in all_data.created_at.apply(lambda x: pd.to_datetime(x).date()).unique():
        company_reply_time[date]= {"mean_reply_time": mean_reply_time, "counts": counts}

    for index, row in company_tweets.iterrows():
        initial_tweet_id = int(row["in_reply_to_status_id"])
        try:
            initial_tweet = all_data.loc[initial_tweet_id]
            date = pd.to_datetime(initial_tweet["created_at"]).date()
            reply_time = row["created_at"] - initial_tweet["created_at"]
            #counts += 1
            #mean_reply_time = calculate_mean(reply_time, mean_reply_time, counts)
            company_reply_time[date]["counts"] += 1
            company_reply_time[date]["mean_reply_time"] = calculate_mean(reply_time, company_reply_time[date]["mean_reply_time"], company_reply_time[date]["counts"])
        except KeyError:
            pass
    
    reply_times[company] = company_reply_time
    #reply_times.append((company, mean_reply_time, counts))

#reply_time_data = pd.DataFrame(reply_times, columns=["company", "mean_reply_time", "nr_of_replies"])


In [34]:
pp.pprint(reply_times) 

{   'AerLingus': {   datetime.date(2020, 3, 10): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetime.date(2020, 3, 11): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetime.date(2020, 3, 12): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetime.date(2020, 3, 13): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetime.date(2020, 3, 14): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetime.date(2020, 3, 15): {   'counts': 0,
                                                     'mean_reply_time': datetime.timedelta(0)},
                     datetim

In [48]:
reply_time_data.head(20)

,company,mean_reply_time,nr_of_replies
0,lufthansa,0 days 01:45:29.332173,575
1,airfrance,1 days 07:53:09.021276,141
2,British_Airways,0 days 22:54:09.387267,754
3,AerLingus,0 days 04:52:45.320000,575
4,vueling,0 days 03:24:15.226890,119
5,Ryanair,0 days 04:33:20.446009,213
6,aeroflot,0 days 02:12:58.377358,53
7,TurkishAirlines,0 days 00:00:00,0
8,easyJet,1 days 00:58:20.767164,670
9,SAS,0 days 03:57:20.659574,235


In [ ]:
#Dataframe med tweets grupperade efter företag och datum, med avg sentiment för varje datum. 

#Kopierar datan till en ny df 
mean_sentiment= all_data.copy()
#Gör om tiden till specifika datum. Kan behöva kommenteras bort om redan körd en gång
mean_sentiment["created_at"] = pd.to_datetime(mean_sentiment['created_at']).dt.to_period("D") 
#Grupperar datan efter företag och datum, samt beräknar avg sentiment. 
mean_sentiment=mean_sentiment.groupby(['associated_company','created_at'])['sentiment'].mean()


In [ ]:
#Plotta avg sentimentet för varje datum och företag

fig, axes = plt.subplots(18, sharey=True, figsize=(10,25))

mean_sentiment.unstack(level=0).plot(kind='bar', subplots=True, ax=axes)

for ax in axes[:-1]:
    ax.set_xticks([]) 
    ax.set_xlabel('') 
    ax.set_title('')
    ax.axhline(linewidth=0.5, color='b',ls='--' )
    ax.legend(loc=9) 

axes[-1].legend(loc=9) 
axes[-1].axhline(linewidth=0.5, color='b',ls='--' )
axes[-1].set_title('')

In [39]:
#          Dataframe där all analysdata läggs in
#------------------------------------------------------------------------

# KOLUMNER: 

#avg_sentiment = det genomsnittliga sentimentet av alla tweets relaterade till ett företag för ett specifikt datum.

#avg_customer_tweet_sentiment = det genomsnittliga sentimentet av kunders initialtweets riktade till ett företag, för ett specifikt datum 
#customer_tweet_count = antalet initiala tweets riktade till företaget

#avg_customer_reply_sentiment = det genomsnittliga sentimentet av trådskaparens svar på ett företags svar, för ett specifikt datum
#avg_customer_change = den genomsnittliga förändringen i sentimentet från kundens första tweet, till dess svar på företagets svar
#customer_reply_count = antalet svar gjorda av trådskaparen, på företagsvar av deras initialtweet
#customer_tweet_std = standardavvikelsen för sentimentet på kunders initialtweets 
#customer_reply_std = standardavvikelsen för sentimentet på trådskaparens svar på företagssvar

#(Denna är lite rörig, vet inte riktigt vad jag själv menar med företagstweets - om det är företags initiala tweets eller företagssvar på kunders tweets eller båda två)
#avg_customers_replies_sentiment = det genomsnittliga sentimentet av alla kunders svar på ett företags tweet, för ett specifikt datum
#avg_customers_change = den genomsnittliga skillnaden på kunders initiala tweet och allas svar på företagstweets
#customer_replies_count = analet kundsvar gjorda på företagstweets 
#customer_replies_std = standardavvikelsen för sentimentet på svar på företagstweets

#avg_company_tweet_sentiment = genomsnittligt sentiment av företags initialtweets
#avg_company_reply_sentiment = genomsnittligt sentiment av företagssvar på kunders tweets
#company_tweets_count = antalet initialtweets gjorde av ett företag
#company_replies_count = antalet företagssvar på kunder tweets
#company_replies/customer_tweets = antalet företagssvar per kunders initialtweets
#avg_company_reply_time = genomsnittlig svarstid för företag på kunders tweets 

COLUMNS = ["date", "company", "avg_sentiment", "avg_customer_tweet_sentiment", "customer_tweet_count", "customer_tweet_std", "avg_customer_reply_sentiment", "avg_customer_change", "customer_reply_count", "customer_reply_std", "avg_customers_replies_sentiment", "avg_customers_change", "customers_replies_count",   "customers_replies_std", "company_tweet_sentiment", "company_reply_sentiment","company_tweets_count", "company_replies_count", "company_replies/customer_tweets", "avg_company_reply_time"]

analysed_data = pd.DataFrame(columns=COLUMNS)